# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0        80.0         69.0  21453.0          6.0          5.0     1969.0   
1        60.0         79.0  12420.0          7.0          5.0     2001.0   
2        20.0         75.0   9742.0          8.0          5.0     2002.0   
3       120.0         39.0   5389.0          8.0          5.0     1995.0   
4        60.0         85.0  11003.0         10.0          5.0     2008.0   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  ...  SaleType_ConLw  \
0        1969.0         0.0       938.0         0.0  ...             0.0   
1        2001.0         0.0       666.0         0.0  ...             0.0   
2        2002.0       281.0         0.0         0.0  ...             0.0   
3        1996.0         0.0      1180.0         0.0  ...             0.0   
4        2008.0       160.0       765.0         0.0  ...             0.0   

   SaleType_New  SaleType_Oth  SaleType_WD  SaleCondition_Abnorml  \
0           0.0           0.0          1.0                    0.0   
1           0.0           0.0          1.0                    0.0   
2           0.0           0.0          1.0                    0.0   
3           0.0           0.0          1.0                    0.0   
4           0.0           0.0          1.0                    0.0   

   SaleCondition_AdjLand  SaleCondition_Alloca  SaleCondition_Family  \
0                    0.0                   0.0                   0.0   
1                    0.0                   0.0                   0.0   
2                    0.0                   0.0                   0.0   
3                    0.0                   0.0                   0.0   
4                    0.0                   0.0                   0.0   

   SaleCondition_Normal  SaleCondition_Partial  
0                   1.0                    0.0  
1                   1.0                    0.0  
2                   1.0                    0.0  
3                   1.0                    0.0  
4                   1.0                    0.0  

[5 rows x 296 columns]

## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, 
                                activation='relu', 
                                input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 272us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
1051/1051 [==============================] - 0s 52us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
1051/1051 [==============================] - 0s 52us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - 

1051/1051 [==============================] - 0s 57us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
1051/1051 [==============================] - 0s 52us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150


1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), 
                              columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), 
                            columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), 
                             columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model 
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 230us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
1051/1051 [==============================] - 0s 44us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - 

1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
1051/1051 [==============================] - 0s 44us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
1051/1051 [==============================] - 0s 44us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150


1051/1051 [==============================] - 0s 44us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [9]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled =  ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [10]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 180us/step - loss: 0.5035 - mse: 0.5035 - val_loss: 0.2219 - val_mse: 0.2219
Epoch 2/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.2178 - mse: 0.2178 - val_loss: 0.1752 - val_mse: 0.1752
Epoch 3/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.1802 - mse: 0.1802 - val_loss: 0.1624 - val_mse: 0.1624
Epoch 4/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1618 - mse: 0.1618 - val_loss: 0.1394 - val_mse: 0.1394
Epoch 5/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.1465 - mse: 0.1465 - val_loss: 0.1311 - val_mse: 0.1311
Epoch 6/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.1325 - mse: 0.1325 - val_loss: 0.1258 - val_mse: 0.1258
Epoch 7/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.1259 - mse: 0.1259 - val_loss: 0.1265 - val_mse: 0.1265


1051/1051 [==============================] - 0s 43us/step - loss: 0.0273 - mse: 0.0273 - val_loss: 0.1364 - val_mse: 0.1364
Epoch 61/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0276 - mse: 0.0276 - val_loss: 0.1342 - val_mse: 0.1342
Epoch 62/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0265 - mse: 0.0265 - val_loss: 0.1399 - val_mse: 0.1399
Epoch 63/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0271 - mse: 0.0271 - val_loss: 0.1420 - val_mse: 0.1420
Epoch 64/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0269 - mse: 0.0269 - val_loss: 0.1384 - val_mse: 0.1384
Epoch 65/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0257 - mse: 0.0257 - val_loss: 0.1377 - val_mse: 0.1377
Epoch 66/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0249 - mse: 0.0249 - val_loss: 0.1393 - val_mse: 0.1393
Epoch 67/150
1051/1051 [==============================

1051/1051 [==============================] - 0s 43us/step - loss: 0.0139 - mse: 0.0139 - val_loss: 0.1485 - val_mse: 0.1485
Epoch 120/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0139 - mse: 0.0139 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 121/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1486 - val_mse: 0.1486
Epoch 122/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0139 - mse: 0.0139 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 123/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0134 - mse: 0.0134 - val_loss: 0.1497 - val_mse: 0.1497
Epoch 124/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1480 - val_mse: 0.1480
Epoch 125/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.1549 - val_mse: 0.1549
Epoch 126/150
1051/1051 [=======================

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [11]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 19us/step


[0.010192078451398268, 0.010192078538239002]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [12]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 27us/step


[0.15452296414302782, 0.15452297031879425]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [13]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

30890.718741989418

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [14]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100,
                          kernel_initializer='he_normal',
                          activation='relu',
                          input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD',
                 loss='mse',
                 metrics=['mse'])

# Train the model
he_model.fit(X_train,
             y_train_scaled,
             batch_size=32,
             epochs=150,
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 183us/step - loss: 0.5880 - mse: 0.5880 - val_loss: 0.2512 - val_mse: 0.2512
Epoch 2/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.2799 - mse: 0.2799 - val_loss: 0.1970 - val_mse: 0.1970
Epoch 3/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.2158 - mse: 0.2158 - val_loss: 0.2011 - val_mse: 0.2011
Epoch 4/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.1875 - mse: 0.1875 - val_loss: 0.1556 - val_mse: 0.1556
Epoch 5/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1613 - mse: 0.1613 - val_loss: 0.1509 - val_mse: 0.1509
Epoch 6/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1408 - mse: 0.1408 - val_loss: 0.1404 - val_mse: 0.1404
Epoch 7/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1345 - mse: 0.1345 - val_loss: 0.1568 - val_mse: 0.1568


1051/1051 [==============================] - 0s 53us/step - loss: 0.0269 - mse: 0.0269 - val_loss: 0.1467 - val_mse: 0.1467
Epoch 61/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.1482 - val_mse: 0.1482
Epoch 62/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0263 - mse: 0.0263 - val_loss: 0.1494 - val_mse: 0.1494
Epoch 63/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.1497 - val_mse: 0.1497
Epoch 64/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.1477 - val_mse: 0.1477
Epoch 65/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0252 - mse: 0.0252 - val_loss: 0.1474 - val_mse: 0.1474
Epoch 66/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1480 - val_mse: 0.1480
Epoch 67/150
1051/1051 [==============================

1051/1051 [==============================] - 0s 52us/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1483 - val_mse: 0.1483
Epoch 120/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1490 - val_mse: 0.1490
Epoch 121/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.1482 - val_mse: 0.1482
Epoch 122/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0125 - mse: 0.0125 - val_loss: 0.1484 - val_mse: 0.1484
Epoch 123/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1482 - val_mse: 0.1482
Epoch 124/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.1490 - val_mse: 0.1490
Epoch 125/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.1485 - val_mse: 0.1485
Epoch 126/150
1051/1051 [=======================

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 22us/step


[0.008822414698832439, 0.008822414092719555]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 30us/step


[0.1517429280417047, 0.15174292027950287]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [17]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, 
                             kernel_initializer='lecun_normal', 
                             activation='relu', 
                             input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 195us/step - loss: 0.5336 - mse: 0.5336 - val_loss: 0.1963 - val_mse: 0.1963
Epoch 2/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.2333 - mse: 0.2333 - val_loss: 0.1489 - val_mse: 0.1489
Epoch 3/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.1834 - mse: 0.1834 - val_loss: 0.1503 - val_mse: 0.1503
Epoch 4/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.1572 - mse: 0.1572 - val_loss: 0.1155 - val_mse: 0.1155
Epoch 5/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.1378 - mse: 0.1378 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 6/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.1208 - mse: 0.1208 - val_loss: 0.1118 - val_mse: 0.1118
Epoch 7/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.1137 - mse: 0.1137 - val_loss: 0.1150 - val_mse: 0.1150


1051/1051 [==============================] - 0s 51us/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 61/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 62/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1060 - val_mse: 0.1060
Epoch 63/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.1063 - val_mse: 0.1063
Epoch 64/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 65/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1045 - val_mse: 0.1045
Epoch 66/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1098 - val_mse: 0.1098
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1064 - val_mse: 0.1064
Epoch 121/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 122/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 123/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 124/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1032 - val_mse: 0.1032
Epoch 125/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 126/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 127/150
1051/1051 [=========

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 24us/step


[0.0074368760128860865, 0.007436876185238361]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 30us/step


[0.10566670082141238, 0.10566669702529907]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [20]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(
    100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop',
                      loss='mse',
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train,
                  y_train_scaled,
                  batch_size=32,
                  epochs=150,
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 233us/step - loss: 0.3423 - mse: 0.3423 - val_loss: 0.1017 - val_mse: 0.1017
Epoch 2/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.1720 - mse: 0.1720 - val_loss: 0.0964 - val_mse: 0.0964
Epoch 3/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.1241 - mse: 0.1241 - val_loss: 0.1044 - val_mse: 0.1044
Epoch 4/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0983 - mse: 0.0983 - val_loss: 0.1056 - val_mse: 0.1056
Epoch 5/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.0820 - mse: 0.0820 - val_loss: 0.1194 - val_mse: 0.1194
Epoch 6/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.0655 - mse: 0.0655 - val_loss: 0.0953 - val_mse: 0.0953
Epoch 7/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0583 - mse: 0.0583 - val_loss: 0.1013 - val_mse: 0.1013


1051/1051 [==============================] - 0s 58us/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.0858 - val_mse: 0.0858
Epoch 61/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.0875 - val_mse: 0.0875
Epoch 62/150
1051/1051 [==============================] - 0s 60us/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.0957 - val_mse: 0.0957
Epoch 63/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.0953 - val_mse: 0.0953
Epoch 64/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.0886 - val_mse: 0.0886
Epoch 65/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.0930 - val_mse: 0.0930
Epoch 66/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0082 - mse: 0.0082 - val_loss: 0.1011 - val_mse: 0.1011
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0075 - mse: 0.0075 - val_loss: 0.0878 - val_mse: 0.0878
Epoch 121/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.0834 - val_mse: 0.0834
Epoch 122/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0074 - mse: 0.0074 - val_loss: 0.0984 - val_mse: 0.0984
Epoch 123/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0032 - mse: 0.0032 - val_loss: 0.0841 - val_mse: 0.0841
Epoch 124/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.0815 - val_mse: 0.0815
Epoch 125/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0873 - val_mse: 0.0873
Epoch 126/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0846 - val_mse: 0.0846
Epoch 127/150
1051/1051 [=========

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 23us/step


[0.004633185520615752, 0.00463318545371294]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 34us/step


[0.09032500787379624, 0.0903250128030777]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [23]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam',
                   loss='mse',
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train,
               y_train_scaled,
               batch_size=32,
               epochs=150,
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 273us/step - loss: 0.4436 - mse: 0.4436 - val_loss: 0.2019 - val_mse: 0.2019
Epoch 2/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.1611 - mse: 0.1611 - val_loss: 0.1317 - val_mse: 0.1317
Epoch 3/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.1085 - mse: 0.1085 - val_loss: 0.1215 - val_mse: 0.1215
Epoch 4/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0761 - mse: 0.0761 - val_loss: 0.1167 - val_mse: 0.1167
Epoch 5/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0639 - mse: 0.0639 - val_loss: 0.1255 - val_mse: 0.1255
Epoch 6/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0472 - mse: 0.0472 - val_loss: 0.1197 - val_mse: 0.1197
Epoch 7/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0396 - mse: 0.0396 - val_loss: 0.1279 - val_mse: 0.1279


1051/1051 [==============================] - 0s 62us/step - loss: 0.0032 - mse: 0.0032 - val_loss: 0.1080 - val_mse: 0.1080
Epoch 61/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.1118 - val_mse: 0.1118
Epoch 62/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.1059 - val_mse: 0.1059
Epoch 63/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1131 - val_mse: 0.1131
Epoch 64/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 65/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 66/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0010 - mse: 0.0010 - val_loss: 0.1105 - val_mse: 0.1105
Epoch 67/150
1051/1051 [==============================

Epoch 119/150
1051/1051 [==============================] - 0s 65us/step - loss: 3.5741e-04 - mse: 3.5741e-04 - val_loss: 0.1043 - val_mse: 0.1043
Epoch 120/150
1051/1051 [==============================] - 0s 67us/step - loss: 5.4510e-04 - mse: 5.4510e-04 - val_loss: 0.1010 - val_mse: 0.1010
Epoch 121/150
1051/1051 [==============================] - 0s 65us/step - loss: 7.6692e-04 - mse: 7.6692e-04 - val_loss: 0.1038 - val_mse: 0.1038
Epoch 122/150
1051/1051 [==============================] - 0s 63us/step - loss: 9.2438e-04 - mse: 9.2438e-04 - val_loss: 0.1016 - val_mse: 0.1016
Epoch 123/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1093 - val_mse: 0.1093
Epoch 124/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0027 - mse: 0.0027 - val_loss: 0.1025 - val_mse: 0.1025
Epoch 125/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.1070 - val_mse: 0.1070
Ep

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 23us/step


[0.0006450085857968164, 0.0006450085784308612]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 30us/step


[0.10041288538577439, 0.10041288286447525]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [26]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 34us/step


[0.14618439380436726, 0.14618439972400665]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [27]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

30045.676236311352

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.